In [1]:
import sys
import os
import torch
from torch import optim
from time import time

In [2]:
tile2vec_dir = '/atlas/u/swang/software/GitHub/tile2vec'
sys.path.append('../')
sys.path.append(tile2vec_dir)

In [3]:
from src.datasets import TileTripletsDataset, GetBands, RandomFlipAndRotate, ClipAndScale, ToFloatTensor, triplet_dataloader
from src.tilenet import make_tilenet

In [4]:
from src.training import prep_triplets, train_triplet_epoch

# Step 1. Download triplets from bucket

Using the download link, unzip triplets into the directory /tile2vec/data/triplets.

# Step 2. Set up dataloader

In [5]:
# Environment stuff
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda = torch.cuda.is_available()


Set up the dataloader for training.

In [6]:
# Change these arguments to match your directory and desired parameters
img_type = 'naip'
tile_dir = '/Users/calummcmeekin/Documents/tile2vec-master/data/triplets/'
bands = 4
augment = True
batch_size = 50
shuffle = True
num_workers = 4
n_triplets = 100000

In [7]:
dataloader = triplet_dataloader(img_type, tile_dir, bands=bands, augment=augment,
                                batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, 
                                n_triplets=n_triplets, pairs_only=True)
print('Dataloader set up complete.')

Dataloader set up complete.


# Step 3. Set up TileNet

In [8]:
in_channels = bands
z_dim = 512

In [9]:
TileNet = make_tilenet(in_channels=in_channels, z_dim=z_dim)
TileNet.train()
if cuda: TileNet.cuda()
print('TileNet set up complete.')

TileNet set up complete.


Set up optimizer.

In [10]:
lr = 1e-3
optimizer = optim.Adam(TileNet.parameters(), lr=lr, betas=(0.5, 0.999))

# Step 4. Train model!

In [11]:
epochs = 1
margin = 10
l2 = 0.01
print_every = 1000
save_models = True


Define the directory for saving models.

In [12]:
model_dir = '/Users/calummcmeekin/Documents/tile2vec-master/models'
if not os.path.exists(model_dir): os.makedirs(model_dir)

In [13]:
t0 = time()
with open("results_fn", 'w') as file:

    print('Begin training.................')
    for epoch in range(0, epochs):
        print (epoch)
        (avg_loss, avg_l_n, avg_l_d, avg_l_nd) = train_triplet_epoch(
            TileNet, cuda, dataloader, optimizer, epoch+1, margin=margin, l2=l2,
            print_every=print_every, t0=t0)

Begin training.................
0
Epoch 1: [1000/100000 (1%)], Avg loss: 8.5715


KeyboardInterrupt: 

In [ ]:
# Save model after last epoch
if save_models:
    model_fn = os.path.join(model_dir, 'TileNet_epoch50.ckpt')
    torch.save(TileNet.state_dict(), model_fn)